In [13]:
import cv2
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Step 1: Data Preparation
train_dir = 'train/'
test_dir = 'test/'
validation_dir = 'validation/'

# Step 2: Feature Extraction (SIFT)
def extract_sift_features_from_folder(folder):
    features = []
    labels = []
    for class_name in os.listdir(folder):
        class_dir = os.path.join(folder, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            sift = cv2.SIFT_create()
            keypoints, descriptors = sift.detectAndCompute(image, None)
            if descriptors is not None and len(descriptors) > 0:
                features.extend(descriptors)  # Extend the features list with descriptors from the current image
                labels.extend([class_name] * len(descriptors))  # Append the corresponding label for each descriptor
    return features, labels


# Extract SIFT features and labels separately for train, test, and validation sets
X_train_features, y_train = extract_sift_features_from_folder(train_dir)
X_test_features, y_test = extract_sift_features_from_folder(test_dir)
X_val_features, y_val = extract_sift_features_from_folder(validation_dir)

# Filter out images that don't have any SIFT features
X_train_filtered = [feat for feat in X_train_features if len(feat) > 0]
X_test_filtered = [feat for feat in X_test_features if len(feat) > 0]
X_val_filtered = [feat for feat in X_val_features if len(feat) > 0]

# Print the number of samples before and after filtering
print("Number of train samples before filtering:", len(X_train_features))
print("Number of train samples after filtering:", len(X_train_filtered))

# Concatenate all feature arrays into a single numpy array
X_train = np.vstack(X_train_filtered)
X_test = np.vstack(X_test_filtered)
X_val = np.vstack(X_val_filtered)

# Ensure the number of labels matches the number of samples
assert len(X_train) == len(y_train), "Number of train samples and labels do not match"
assert len(X_test) == len(y_test), "Number of test samples and labels do not match"
assert len(X_val) == len(y_val), "Number of validation samples and labels do not match"


# Step 3: Training
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4: Validation
y_val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Step 5: Testing
y_test_pred = clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

import pickle
with open('SIFT with RF_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

Number of train samples before filtering: 53076
Number of train samples after filtering: 53076
Validation Accuracy: 0.8714724490063791
Test Accuracy: 0.5482594133766365


In [26]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import pickle

# Function to extract SIFT features from an image
def extract_sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

# Function to classify an image using a pre-trained model
def classify_image(image, clf):
    features = extract_sift_features(image)
    if features is not None:
        prediction = clf.predict(features)
        return prediction
        
    else:
        return None
 
# Function to open an image file and classify it
def open_and_classify_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        prediction = classify_image(image, clf)

        if prediction is not None:
            result_label.config(text=f"Prediction: {prediction}")
        else:
            messagebox.showerror("Error", "No SIFT features found in the image")
 
    
# Load pre-trained classifier from pickle file
with open('SIFT with RF_classifier.pkl', 'rb') as file:
    clf = pickle.load(file)

# Create the main application window
app = tk.Tk()
app.title("SIFT Image Classification")

# Create a button to open an image file
open_button = tk.Button(app, text="Open Image", command=open_and_classify_image)
open_button.pack(pady=10)

# Create a label to display the classification result
result_label = tk.Label(app, text="")
result_label.pack(pady=10)


# Run the application
app.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import pickle

# Load pre-trained classifier from pickle file
with open('classifier.pkl', 'rb') as file:
    clf = pickle.load(file)

# Function to extract SIFT features from an image
def extract_sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

# Function to classify an image using a pre-trained model
def classify_image(image):
    features = extract_sift_features(image)
    if features is not None:
        prediction = clf.predict(features)
        # Convert prediction array to a set to remove duplicate labels
        unique_predictions = set(prediction)
        return list(unique_predictions)  # Convert set back to a list
    else:
        return None

# Function to open an image file, classify it, and display the result
def open_and_classify_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        predictions = classify_image(image)
        if predictions is not None:
            if len(predictions) == 1:
                messagebox.showinfo("Prediction", f"Class Label: {predictions[0]}")
            else:
                messagebox.showinfo("Predictions", f"Class Labels: {', '.join(predictions)}")
        else:
            messagebox.showerror("Error", "No SIFT features found in the image")

# Create the main application window
app = tk.Tk()
app.title("SIFT Image Classification")

# Create a button to open an image file and classify it
open_button = tk.Button(app, text="Open Image", command=open_and_classify_image)
open_button.pack(pady=10)

# Run the application
app.mainloop()
